# Add sidewalk widths to basic pedestrian network

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd

import shapely.ops as so
import shapely.geometry as sg
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline

import networkx as nx
import momepy

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import poly_utils
import bgt_utils

import matplotlib.pyplot as plt
import folium

import settings as st
if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

## Import basic network data

In [ ]:
# Get basic pedestrian network and sidewalks
df_bgt_nw = gpd.read_file(cf.output_basic_network)
df_bgt_sw = gpd.read_file(cf.output_sidewalks_basic_network)

In [ ]:
df_bgt_nw.rename(columns={'geometry': 'centerlines'}, inplace=True)
df_bgt_exp = pd.merge(df_bgt_nw, df_bgt_sw, on='sidewalk_id', how='left')
df_bgt_exp = df_bgt_exp.set_geometry('centerlines')
df_bgt_exp.head(3)

## Create relevant variable - full width

### Get width and color

In [ ]:
df_bgt_exp[['avg_width', 'min_width']] = df_bgt_exp.progress_apply(
    lambda row: poly_utils.get_avg_width_cl(row.geometry, row.centerlines, 
                                            st.width_resolution, st.width_precision), axis=1)

In [ ]:
conditions = [
    (df_bgt_exp['min_width'] < st.width_1),
    (df_bgt_exp['min_width'] >= st.width_1) & (df_bgt_exp['min_width'] < st.width_2),
    (df_bgt_exp['min_width'] >= st.width_2) & (df_bgt_exp['min_width'] < st.width_3),
    (df_bgt_exp['min_width'] >= st.width_3) & (df_bgt_exp['min_width'] < st.width_4),
    (df_bgt_exp['min_width'] >= st.width_4) & (df_bgt_exp['min_width'] < st.width_5),
    (df_bgt_exp['min_width'] >= st.width_5) & (df_bgt_exp['min_width'] < st.width_6),
    (df_bgt_exp['min_width'] >= st.width_6)
]

values_color = ['darkred', 'red', 'orange', 'yellow', 'greenyellow', 'limegreen', 'green']
values_indication = ['<' + str(st.width_1) + 'm', 
                     str(st.width_1) + '-' + str(st.width_2) + 'm', 
                     str(st.width_2) + '-' + str(st.width_3) + 'm', 
                     str(st.width_3) + '-' + str(st.width_5) + 'm', 
                     str(st.width_4) + '-' + str(st.width_5) + 'm', 
                     str(st.width_5) + '-' + str(st.width_6) + 'm', 
                     '>' + str(st.width_6) + 'm']

In [ ]:
df_bgt_exp['full_width_color'] = np.select(conditions, values_color)
df_bgt_exp['full_width'] = np.select(conditions, values_indication)
df_bgt_exp['full_width'].value_counts()

### Remove too narrow paths and short-ends

In [ ]:
# Apply minimal path width on BGT centerlines
print(df_bgt_exp.shape)
df_bgt_exp = df_bgt_exp[df_bgt_exp['min_width'] > st.min_path_width].reset_index(drop=True)
print(df_bgt_exp.shape)

In [ ]:
# Remove short lines
df_bgt_exp = df_bgt_exp.rename(columns={'geometry':'geometry_sidewalks', 'centerlines':'geometry'}) 
mls_per_id = poly_utils.create_mls_per_sidewalk(df_bgt_exp, crs=st.CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(
    lambda x: poly_utils.remove_short_lines(x, st.min_se_length_fw))

In [ ]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_bgt_exp = df_bgt_exp.merge(long_segments_df, how='inner')
df_bgt_exp.shape

## Import segments data

In [ ]:
# Read lines with widths (calculated in notebook 5 of https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Sidewalk_Width)
df_segments_full = gpd.read_file(cf.segments_file, crs=st.CRS)

### Selection of segments within pilot area

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

# Only keep BGT data within pilot areas
df_segments = df_segments_full.sjoin(df_areas, how='inner', predicate='within')  # note: only sidewalk polygons fully inside area are included

#del df_segments_full

## Create relevant variable - obstacle-free width

### Remove too narrow paths and short-ends

In [ ]:
# Apply minimal path width
df_segments_wide = df_segments[df_segments['min_width'] > st.min_path_width].reset_index(drop=True)
print(df_segments.shape)
print(df_segments_wide.shape)

In [ ]:
# Remove short lines (this takes a while)
mls_per_id = poly_utils.create_mls_per_sidewalk(df_segments_wide, crs=st.CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(poly_utils.remove_short_lines)

In [ ]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_segments_wide = df_segments_wide.merge(long_segments_df, how='inner')
df_segments_wide.shape

### Use color codes

In [ ]:
conditions = [
    (df_segments_wide['min_width'] < st.width_1),
    (df_segments_wide['min_width'] >= st.width_1) & (df_segments_wide['min_width'] < st.width_2),
    (df_segments_wide['min_width'] >= st.width_2) & (df_segments_wide['min_width'] < st.width_3),
    (df_segments_wide['min_width'] >= st.width_3) & (df_segments_wide['min_width'] < st.width_4),
    (df_segments_wide['min_width'] >= st.width_4) & (df_segments_wide['min_width'] < st.width_5),
    (df_segments_wide['min_width'] >= st.width_5) & (df_segments_wide['min_width'] < st.width_6),
    (df_segments_wide['min_width'] >= st.width_6)
]

values = ['darkred', 'red', 'orange', 'yellow', 'greenyellow', 'limegreen', 'green']

In [ ]:
df_segments_wide['min_width_color'] = np.select(conditions, values)
df_segments_wide['min_width_color'].value_counts()

In [ ]:
# Add width factor, for calculating the weights of the paths later
values = [1000000000000, 10000000000, 100000000, 1000000, 10000, 100, 1]
df_segments_wide['min_width_factor'] = np.select(conditions, values).astype('int64')

### Take point cloud coverage into account

In [ ]:
df_segments_wide['min_width_factor'][df_segments_wide['pc_coverage'] == False] = 100000000000001

In [ ]:
df_segments_wide['min_width_factor'].value_counts()

### Do network calculation

In [ ]:
df_bgt_exp['route_weight'] = np.nan
print('start doing network calculation on ' + str(len(df_bgt_sw['sidewalk_id'])) + ' rows')

In [ ]:
# Create final dataframe
final_df = pd.DataFrame()

for i in range(len(df_bgt_sw['sidewalk_id'])):
    print(i)
      
    # Get sidewalk polygon for this centerline 
    my_sidewalk = df_bgt_sw['geometry'].values[i]   
    
    # Create graph for all paths withing this sidewalk polygon
    df_sidewalk = df_segments_wide[df_segments_wide['geometry'].within(my_sidewalk)].reset_index(drop=True)
    G = momepy.gdf_to_nx(df_sidewalk, approach="primal", multigraph=True)
    
    # Create dataframe with linestrings of centerline
    centerline_df = df_bgt_exp[df_bgt_exp['sidewalk_id'] == i].reset_index(drop=True)
    
    for j in range(len(centerline_df['geometry'])):  
        
        # Get line
        my_line = centerline_df.iloc[[j]]['geometry'].values[0]
        
        if len(list(G.nodes)) > 0:
            # Check if my_line has start and end (not a ring)
            if len(my_line.boundary) == 0:
                print('no route calculated for line (j)', j, 'in sidewalk (i)', i, '(ring)')
                centerline_df['route_weight'][j] = 0
            else:
                # Get origin and destination location
                origin_point, dest_point = my_line.boundary
                origin_node_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G.nodes)))[1]
                dest_node_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G.nodes)))[1]

                # Get origin and destination node
                origin_node = (origin_node_loc.x, origin_node_loc.y)
                dest_node = (dest_node_loc.x, dest_node_loc.y)   

                # Get weight of optimal route in graph
                try:
                    route_weight = nx.shortest_path_length(G, origin_node, dest_node,
                                                           weight='min_width_factor')
                    if (origin_point.distance(origin_node_loc) < st.max_dist) and (dest_point.distance(dest_node_loc) < st.max_dist):
                        centerline_df['route_weight'][j] = route_weight
                    else:
                        print('origin and/or destination node too far from line start/end for line (j)', j, 'in sidewalk (i)', i) 
                        centerline_df['route_weight'][j] = np.nan
                except nx.NetworkXNoPath:
                    print('no route found for line (j)', j, 'in sidewalk (i)', i)
                    centerline_df['route_weight'][j] = 100000000000000
        else:
            print('network has zero nodes')

    # Append data to final dataframe
    final_df = pd.concat([final_df, centerline_df])
final_df = final_df.reset_index()

In [ ]:
#del df_bgt_exp

In [ ]:
# Get final color of the routes
final_df['final_color'] = final_df.progress_apply(
    lambda row: poly_utils.get_route_color(row.route_weight), axis=1)

## Post-process 

In [ ]:
final_df = final_df.set_geometry('geometry')

### Set final color and indication

In [ ]:
# Get dataframe with only valid lines  # TODO now first color, then widths and then delete color --> just use with in the first place
final_df_select = final_df[final_df['final_color'].isin(['green', 'limegreen', 'greenyellow', 
                                                         'yellow', 'orange', 'red',
                                                         'darkred', 'purple', 'black', 'grey'])]

final_df_select = final_df_select.reset_index(drop=True)

In [ ]:
# Recolor paths with unknown widths
final_df_select['final_color'] = final_df_select['final_color'].replace(['black', 'purple'], 'grey')

In [ ]:
# Add meter indication
conditions = [
    (final_df_select['final_color'] == 'darkred'),
    (final_df_select['final_color'] == 'red'),   
    (final_df_select['final_color'] == 'orange'),
    (final_df_select['final_color'] == 'yellow'),
    (final_df_select['final_color'] == 'greenyellow'),
    (final_df_select['final_color'] == 'limegreen'),  
    (final_df_select['final_color'] == 'green'),  
    (final_df_select['final_color'] == 'grey')
]
values_indication = ['<' + str(st.width_1) + 'm', 
                     str(st.width_1) + '-' + str(st.width_2) + 'm', 
                     str(st.width_2) + '-' + str(st.width_3) + 'm', 
                     str(st.width_3) + '-' + str(st.width_4) + 'm', 
                     str(st.width_4) + '-' + str(st.width_5) + 'm', 
                     str(st.width_5) + '-' + str(st.width_6) + 'm', 
                     '>' + str(st.width_6) + 'm',
                    'unknown']
final_df_select['obstacle_free_width'] = np.select(conditions, values_indication)

### Get correct columns

In [ ]:
# Select only relevant columns
final_df_select = final_df_select[['geometry', 'final_color', 'obstacle_free_width', 
                                   'full_width', 'full_width_color']]

### Fix incorrect labeling of obstacle-free width (> full width)

In [ ]:
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['0.9-1.5m', '1.5-2.0m', '2.0-2.2m', '2.2-2.9m', '2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '<0.9m')] = '<0.9m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['1.5-2.0m', '2.0-2.2m', '2.2-2.9m', '2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '0.9-1.5m')] = '0.9-1.5m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['2.0-2.2m', '2.2-2.9m', '2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '1.5-2.0m')] = '1.5-2.0m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['2.2-2.9m', '2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '2.0-2.2m')] = '2.0-2.2m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '2.2-2.9m')] = '2.2-2.9m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'] == '>3.6m') & 
                                       (final_df_select['full_width'] == '2.9-3.6m')] = '2.9-3.6m'

### Adjust crs

In [ ]:
final_df_select = final_df_select.set_crs(st.CRS)  # TODO might be removed?
print('finished post-processing')

## Merge with basic network

In [ ]:
df_combined = pd.merge(df_bgt_nw, final_df_select, how='left', left_on='centerlines', right_on='geometry')

## Store final output

In [ ]:
df_store = df_combined[['cl_id', 'centerlines', 'length', 'sidewalk_id', 'obstacle_free_width']]
df_store = df_store.set_geometry('centerlines').set_crs(st.CRS)
df_store.head()

In [ ]:
df_store.to_file(cf.output_file_widths, driver='GPKG') 
print('stored final data')

## Visualize

In [ ]:
df_store = gpd.read_file(cf.output_file_widths)

In [ ]:
# Set center of map
area_coords = [52.375664816888225, 4.8632280955697995]

# Set to map coordinate reference system
df_store_show = df_store.to_crs("EPSG:4326")

# Create basic map
my_map = folium.Map(
    location=area_coords,
    tiles='cartodbpositron',
    min_zoom=10, max_zoom=20, zoom_start=13,
    zoom_control=True, control_scale=True, control=False
    )

for _, r in df_store_show[df_store_show['obstacle_free_width'] == '<0.9m'].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "darkred"})
    geo_j.add_to(my_map)

for _, r in df_store_show[df_store_show['obstacle_free_width'] == '0.9-1.5m'].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "red"})
    geo_j.add_to(my_map)

for _, r in df_store_show[df_store_show['obstacle_free_width'] == '1.5-2.0m'].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "orange"})
    geo_j.add_to(my_map)

for _, r in df_store_show[df_store_show['obstacle_free_width'] == '2.0-2.2m'].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "yellow"})
    geo_j.add_to(my_map)

for _, r in df_store_show[df_store_show['obstacle_free_width'] == '2.2-2.9m'].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "greenyellow"})
    geo_j.add_to(my_map)

for _, r in df_store_show[df_store_show['obstacle_free_width'] == '2.9-3.6m'].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "limegreen"})
    geo_j.add_to(my_map)

for _, r in df_store_show[df_store_show['obstacle_free_width'] == '>3.6m'].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "green"})
    geo_j.add_to(my_map)

for _, r in df_store_show[df_store_show['obstacle_free_width'].isin(['unknown'])].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "grey"})
    geo_j.add_to(my_map)

for _, r in df_store_show[df_store_show['obstacle_free_width'].isin([None])].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "lightgrey"})
    geo_j.add_to(my_map)    

In [ ]:
# Store map
my_map.save(cf.network_map_widths)